# Example 2: Add Parameter and Introduce Parameter Object

[Add parameter in the refactoring catalog](http://refactoring.com/catalog/addParameter.html).

[Introduce Parameter Object in the refactoring catalog](http://refactoring.com/catalog/introduceParameterObject.html).

---

Some code that returns the best movie.

In [28]:
def best_picture():
    return 'Casablanca'

In [29]:
best_picture()

'Casablanca'

---

Say we want this to vary by year, instead of just returning the best movie of all time.

To add a parameter, first make the new parameter optional by giving it a keyword and defaulting it to `None`. Log a warning when the usage doesn't supply the parameters.

In [52]:
import warnings

def best_picture(year=None):
    if year is None:
        warnings.warn('Must specify "year" for best_picture', DeprecationWarning)
    if year == 2015:
        return 'Spotlight'
    elif year == 2014:
        return 'Birdman'
    # ...
    else:
        return 'Casablanca'

In [53]:
best_picture()  # Expect a warning because year isn't specified

/Users/bslatkin/projects/pycon2016/lib/python3.5/site-packages/ipykernel/__main__.py:5: DeprecationWarning: Must specify "year" for best_picture


'Casablanca'

---

Fix those warnings at all call sites, then make the parameter required.

In [81]:
def best_picture(year):
    if year == 2015:
        return 'Spotlight'
    elif year == 2014:
        return 'Birdman'
    raise ValueError('Must specify "year" for best_picture')

In [82]:
best_picture(2014)

'Birdman'

In [83]:
best_picture()  # Expect an error

TypeError: best_picture() missing 1 required positional argument: 'year'

In [84]:
best_picture(None)  # Expect an error

ValueError: Must specify "year" for best_picture

---

Adding a parameter object is very similar.

In [59]:
winners = {
    2010: "The King's Speech",
    2011: 'The Artist',
    2012: 'Argo',
    2013: '12 Years a Slave',
    2014: 'Birdman',
    2015: 'Spotlight',
}

def best_pictures(start, end):
    found = []
    for i in range(start, end + 1):
        found.append(winners[i])
    return found

In [60]:
best_pictures(2014, 2015)

['Birdman', 'Spotlight']

---

Create a helper class that encapsulates the behavior you want for the new parameter. Require an object of this class as a new keyword value. Make it so the parameter object and the old usage are mutually exclusive and a warning is issued for the old usage.

In [85]:
class Criteria:
    def __init__(self, start_year, end_year):
        self.start_year = start_year
        self.end_year = end_year
    
    def get_years(self):
        return range(self.start_year, self.end_year + 1)
        

def best_pictures(start_or_criteria, deprecated_end=None):
    has_old_usage = deprecated_end is not None
    has_new_usage = isinstance(start_or_criteria, Criteria)

    if not (has_old_usage ^ has_new_usage):
        raise TypeError('May only specify ("start" and "end") or "criteria"')

    if has_new_usage:
        criteria = start_or_criteria
    elif has_old_usage:
        warnings.warn('Must specify "criteria" for best_pictures', DeprecationWarning)
        criteria = Criteria(start_or_criteria, deprecated_end)

    return [winners[year] for year in criteria.get_years()]

In [86]:
best_pictures(Criteria(2014, 2015))  # New usage, no warnings or errors

['Birdman', 'Spotlight']

In [87]:
best_pictures(2014, 2015)  # Expect a warning because criteria isn't used

/Users/bslatkin/projects/pycon2016/lib/python3.5/site-packages/ipykernel/__main__.py:20: DeprecationWarning: Must specify "criteria" for best_pictures


['Birdman', 'Spotlight']

In [88]:
best_pictures(Criteria(2014, 2015), 2015)  # Expect an error because a mix of usage is supplied

TypeError: May only specify ("start" and "end") or "criteria"

---

Fix the warnings at all call sites, then make the parameter object required.

In [89]:
def best_pictures(criteria):
    return [winners[year] for year in criteria.get_years()]

In [90]:
best_pictures(Criteria(2014, 2015))

['Birdman', 'Spotlight']

In [92]:
best_pictures(2014, 2015)  # Expect an error because the function signature changed

TypeError: best_pictures() takes 1 positional argument but 2 were given